In [96]:
import pandas as pd


In [98]:
 df = pd.read_csv("/kaggle/input/medicaltranscriptions/mtsamples.csv" , usecols=['description', 'keywords', "transcription" , "medical_specialty","sample_name", "medical_specialty"])


In [99]:
df.dropna(axis = 0 , inplace = True) 

In [101]:
df["total_description"] = "description : " + df["description"] + " medical specialty : " + df["medical_specialty"] + " sample name : " +  df["sample_name"] + " transcription : " + df["transcription"]

In [102]:
import tensorflow as tf
from transformers import BartTokenizer, TFBartForConditionalGeneration
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the optimizer and loss function
optimizer = Adam(learning_rate=5e-5)
loss = SparseCategoricalCrossentropy(from_logits=True)


In [105]:
# Load pre-trained BART model
model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-base')



All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [107]:
# Load BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')



In [108]:
cnt=0
for i in df['total_description']:
    if(len(i.split())<=2000):
        cnt=cnt+1
print(cnt/len(df['total_description']))

0.9994869163673679


In [109]:
cnt=0
for i in df['keywords']:
    if(len(i.split())<=90):
        cnt=cnt+1
print(cnt/len(df['keywords']))

0.9633145202668035


# Using KERAS


In [110]:
import pandas as pd
import tensorflow as tf

max_input_length = 1000
max_output_length = 150

# Define the batch size and number of epochs for training
batch_size = 4
epochs = 4


In [111]:
def tokenize_inputs_and_outputs(inputs, outputs):
    input_ids = []
    attention_masks = []
    decoder_input_ids = []
    labels = []
    for i in range(len(inputs)):
        input_text = inputs[i]
        output_text = outputs[i]
        input_tokenized = tokenizer.encode(input_text, max_length=max_input_length, padding='max_length', truncation=True, return_tensors='tf')
        output_tokenized = tokenizer.encode(output_text, max_length=max_output_length, padding='max_length', truncation=True, return_tensors='tf')
        input_ids.append(input_tokenized[0])
        #attention_masks.append(input_tokenized['attention_mask'][0])
        #decoder_input_ids.append(output_tokenized['input_ids'][0])
        labels.append(output_tokenized[0])
    return {'input_ids': tf.stack(input_ids),
           # 'attention_mask': tf.stack(attention_masks),
            #'decoder_input_ids': tf.stack(decoder_input_ids),
            'labels': tf.stack(labels)}


In [112]:
import tensorflow as tf
import numpy as np

In [113]:
import re
def clean_highlight(text):
    newText = text.lower()
    newText = re.sub('[^\w\s\d\.]','',newText)
    newText = ' '.join(newText.split())
    newText = '_START_ '+ newText + ' _END_'
    return newText
def clean_body(text):
    #newText = text[3:]
    newText = text.lower()
    newText = re.sub('[^\w\s\d\.]','',newText)
    newText = ' '.join(newText.split())
    tokens = [w for w in newText.split() ]
    long_words=[]
    for i in tokens:
        if len(i)>=3:
            long_words.append(i)   
    return ( " ".join(long_words)).strip()


In [114]:
clean_body(df['total_description'][1] )

'description consult for laparoscopic gastric bypass. medical specialty bariatrics sample name laparoscopic gastric bypass consult transcription past medical history has difficulty climbing stairs difficulty with airline seats tying shoes used public seating and lifting objects off the floor. exercises three times week home and does cardio. has difficulty walking two blocks five flights stairs. difficulty with snoring. has muscle and joint pains including knee pain back pain foot and ankle pain and swelling. has gastroesophageal reflux disease.past surgical history includes reconstructive surgery his right hand years ago. social history currently single. has about ten drinks year. had smoked significantly until several months ago. now smokes less than three cigarettes day.family history heart disease both grandfathers grandmother with stroke and grandmother with diabetes. denies obesity and hypertension other family members.current medications none.allergies allergic penicillin.miscell

In [115]:
clean_highlight(df['keywords'][1])

'_START_ bariatrics laparoscopic gastric bypass weight loss programs gastric bypass atkins diet weight watchers body weight laparoscopic gastric weight loss pounds months weight laparoscopic band loss diets overweight lost _END_'

In [116]:
# Create a dataset from the input and output data
class SummaryDataset(tf.keras.utils.Sequence):
    def __init__(self, inputs, outputs, batch_size):
        self.inputs = inputs
        self.outputs = outputs
        self.batch_size = batch_size
        self.indexes = np.arange(len(inputs))

    def __len__(self):
        return len(self.inputs) // self.batch_size
    def __getitem__(self, indexes):
        indexes = np.random.choice(self.indexes, size=self.batch_size, replace=False)
        inputs_batch = [self.inputs[i] for i in indexes]
        outputs_batch = [self.outputs[i] for i in indexes]
        
        inputs_batch = [clean_body(text) for text in inputs_batch]
        outputs_batch = [clean_highlight(text) for text in outputs_batch]


        #inputs_batch = self.inputs[idx*self.batch_size:(idx+1)*self.batch_size]
        #outputs_batch = self.outputs[idx*self.batch_size:(idx+1)*self.batch_size]
        return tokenize_inputs_and_outputs(inputs_batch, outputs_batch)




In [117]:
# Create the dataloader for the training data
train_dataset = SummaryDataset(df['total_description'][:-1000].tolist(), df['keywords'][:-1000].tolist(), batch_size=batch_size)
val_dataset = SummaryDataset(df['total_description'][-1000:].tolist(), df['keywords'][-1000:].tolist(), batch_size=batch_size)

In [118]:
next(iter(train_dataset))

{'input_ids': <tf.Tensor: shape=(4, 1000), dtype=int32, numpy=
 array([[    0, 42739,  4885, ...,     1,     1,     1],
        [    0, 42739,   235, ...,     1,     1,     1],
        [    0, 42739, 13827, ...,     1,     1,     1],
        [    0, 42739,  3186, ...,     1,     1,     1]], dtype=int32)>,
 'labels': <tf.Tensor: shape=(4, 150), dtype=int32, numpy=
 array([[    0,  1215,  4014, 11328,  1215, 13938,  5966,  2017,  2775,
         17301, 25743,   298, 20436, 24238,  6402,  1803,  4885, 15664,
          4405, 30960,  1925,  1164, 18234, 18362,    23,   225,  1168,
          1168, 25743,   298, 20436, 24238, 18134,  9309,  1215,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1, 

In [119]:
# Compile the model
loss = SparseCategoricalCrossentropy(from_logits=True)


model.compile(optimizer=optimizer, loss=loss)



In [ ]:
# Fine-tune the T5 model
model.fit(train_dataset,validation_data= val_dataset , epochs= 10  )

In [144]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

# Define the maximum length of the generated summaries
max_summary_length = 90

# Define a list of input texts to generate summaries for
input_texts = df["total_description"][-5:].tolist()
output_texts =   df["keywords"][-5:].tolist()
bleu_scores = []
# Generate summaries for the input texts
for i  in range(1 , len(input_texts)):
    # Tokenize the input text
    input_text = input_texts[i]
    output_text= output_texts[i]
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    # Generate the summary using the bart model
    summary_ids = model.generate(input_ids=input_ids,
                                  max_length=max_summary_length,
                                  num_beams=4,
                                  no_repeat_ngram_size=2,
                                  early_stopping=True)

    # Decode the summary
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    bleu_score = corpus_bleu(output_text.split()[:len(summary_text.split())], summary_text.split())
    bleu_scores.append(bleu_score)
    # Print the input text and summary
    print(f"Input: {input_text}")
    print("\n ")
    print(f"Keywords predictied: {summary_text[7:-7]}")
    print(f"Real Keywords:    {output_text}\n \n")


Input: description :  Patient presented to the Bariatric Surgery Service for consideration of laparoscopic Roux-en-Y gastric bypass.  medical specialty :  Bariatrics sample name :  Bariatric Consult - Surgical Weight Loss - 4  transcription : HISTORY OF PRESENT ILLNESS:,  Ms. A is a 55-year-old female who presented to the Bariatric Surgery Service for consideration of laparoscopic Roux-en-Y gastric bypass.  The patient states that she has been overweight for approximately 35 years and has tried multiple weight loss modalities in the past including Weight Watchers, NutriSystem, Jenny Craig, TOPS, cabbage diet, grape fruit diet, Slim-Fast, Richard Simmons, as well as over-the-counter measures without any long-term sustainable weight loss.  At the time of presentation to the practice, she is 5 feet 6 inches tall with a weight of 285.4 pounds and a body mass index of 46.  She has obesity-related comorbidities, which includes hypertension and hypercholesterolemia.,PAST MEDICAL HISTORY:,  Si

In [143]:

bleu_scores

print("bleu_score =" , bleu_scores[1:])

bleu_score = [0.6573046324907937, 0.6918912876154527, 0.5806030754582867]


In [129]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Function to listen and convert speech to text with pause detection
def listen_and_convert():
    with sr.Microphone() as source:
        print("Speak now...")
        # Listen for audio with a pause threshold of 1 second
        audio = recognizer.listen(source, phrase_time_limit=5)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand what you said."
    except sr.RequestError as e:
        return "Could not request results from Google Speech Recognition service; {0}".format(e)


In [57]:
import pickle

# Save the model
with open('model2.pkl', 'wb') as f:
    pickle.dump(model, f)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...bias_layer
......vars
.........0
...layers
......tf_bart_main_layer
.........decoder
............dropout
...............vars
............embed_positions
...............vars
..................0
............embed_tokens
...............vars
..................0
............layernorm_embedding
...............vars
..................0
..................1
............layers
...............tf_bart_decoder_layer
..................activation_dropout
.....................vars
..................dropout
.....................vars
..................encoder_attn
.....................dropout
........................vars
.....................k_proj
........................vars
...........................0
...........................1
.....................out_proj
........................vars
...........................0
...........................1
.....................q_proj
........................vars
...........................0
...

In [18]:
import pickle

# load the saved model
with open('model2.pkl', 'rb') as f:
    model2 = pickle.load(f)

# use the model for prediction or other tasks


FileNotFoundError: [Errno 2] No such file or directory: 'model2.pkl'

In [1]:
# Define the maximum length of the generated summaries
max_summary_length = 90

# Define a list of input texts to generate summaries for
input_texts = df["description"][:2].tolist()

# Generate summaries for the input texts
for input_text in input_texts:
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    # Generate the summary using the T5 model
    summary_ids = model.generate(input_ids=input_ids,
                                  max_length=max_summary_length,
                                  num_beams=4,
                                  no_repeat_ngram_size=2,
                                  early_stopping=True)

    # Decode the summary
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Print the input text and summary
    print(f"Input: {input_text}")
    print(f"Summary: {summary_text}\n")


NameError: name 'df' is not defined

In [80]:
model.summary()

Model: "tf_bart_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [94]:
input_ids = tokenizer.encode_plus(input_text, return_tensors='tf')


In [36]:
!pip install kaggle


In [26]:
import keras 
print(transformers.__version__)


NameError: name 'transformers' is not defined